In [1]:
import requests
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import cot_reports as cot
import MetaTrader5 as mt5
mt5.initialize()
# Replace following with your MT5 Account Login
account=51434456 # 
password="9UpBvVzc"
server = 'ICMarkets-Demo'

In [61]:
def get_rates(pair1, timeframe, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1[['time','open', 'high', 'low', 'close']].set_index('time')
def get_rates_w(pair1, timeframe, x):
    
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    
    pair1['time'] = pd.to_datetime(pair1['time'], unit='s')
    pair1 = pair1[['time', 'open', 'high', 'low', 'close']].set_index('time')
    
    # Resample to weekly data starting from the day after COT report release
    weekly_data = pair1.resample('W-TUE', label='left').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last'
    })

    return weekly_data

In [5]:
df = pd.DataFrame()
begin_year = 2015
end_year = 2023
for i in range(begin_year, end_year + 1):
    single_year = pd.DataFrame(cot.cot_year(i, cot_report_type='legacy_futopt')) 
    df = pd.concat([df, single_year], axis = 0) 

Selected: legacy_futopt
Downloaded single year data from: 2015
Selected: legacy_futopt
Downloaded single year data from: 2016
Selected: legacy_futopt
Downloaded single year data from: 2017
Selected: legacy_futopt
Downloaded single year data from: 2018
Selected: legacy_futopt
Downloaded single year data from: 2019
Selected: legacy_futopt
Downloaded single year data from: 2020
Selected: legacy_futopt
Downloaded single year data from: 2021
Selected: legacy_futopt
Downloaded single year data from: 2022
Selected: legacy_futopt
Downloaded single year data from: 2023


In [6]:
lst = ['SWISS FRANC', 'BRITISH POUND STERLING', 'JAPANESE YEN', 'EURO FX', 'NEW ZEALAND DOLLAR', 'AUSTRALIAN DOLLAR', 
       'EURO FX/JAPANESE YEN XRATE']

In [7]:
t1 = []
t2 = []
temp = df.set_index('Market and Exchange Names').T
for col in temp.columns.unique():
    if 'CHICAGO MERCANTILE EXCHANGE' in col:
        t1.append(col)
        
for sym in lst:
    for instrument in t1:
        if sym in instrument:
            # print(f"match found on {sym} and {instrument}")
            t2.append(instrument)
selected_cols = temp[t2]

In [8]:
fields_to_drop = [
    "Contract Units",
    "CFTC Contract Market Code (Quotes)",
    "CFTC Market Code in Initials (Quotes)",
    "CFTC Commodity Code (Quotes)",
    "CFTC Contract Market Code",
    "CFTC Market Code in Initials",
    "CFTC Region Code",
    "CFTC Commodity Code", 
    
]

In [9]:
df_dropped = selected_cols.T.drop(columns = fields_to_drop, errors = 'ignore')
df_dict = dict(df_dropped.T)

In [10]:
for name, df in df_dict.items():
    # df[name]
    df = df_dict[name]
    df = df_dict[name].T.reset_index()
    df = df.rename(columns = {'As of Date in Form YYYY-MM-DD' : 'Date'})
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace = True)
    df = df.drop(columns = ['Market and Exchange Names', 'As of Date in Form YYMMDD'])
    df_dict[name] = df
    df_dict[name].sort_index(inplace = True)

### Extra data functions

In [ ]:
df_dict_test = df_dict

In [ ]:
keys_to_remove = []
# Assuming df_dict is your dictionary
for key in df_dict.keys():
    # Check if the key without the trailing space exists
    if key.endswith(' ') and key.rstrip() in df_dict:
        # Concatenate the DataFrame with trailing space (2005-2014) to the one without (2015-2023)
        df_dict[key.rstrip()] = pd.concat([df_dict[key], df_dict[key.rstrip()]])

        # Mark the key with trailing space for removal
        keys_to_remove.append(key)

# Remove the DataFrames with keys that have trailing spaces
for key in keys_to_remove:
    del df_dict[key]

In [138]:
len(df_dict_test['SWISS FRANC - CHICAGO MERCANTILE EXCHANGE'])

989

### Analysis

In [59]:
df_dict['SWISS FRANC - CHICAGO MERCANTILE EXCHANGE']

,Open Interest (All),Noncommercial Positions-Long (All),Noncommercial Positions-Short (All),Noncommercial Positions-Spreading (All),Commercial Positions-Long (All),Commercial Positions-Short (All),Total Reportable Positions-Long (All),Total Reportable Positions-Short (All),Nonreportable Positions-Long (All),Nonreportable Positions-Short (All),...,Concentration-Net LT =8 TDR-Long (Old),Concentration-Net LT =8 TDR-Short (Old),Concentration-Gross LT =4 TDR-Long (Other),Concentration-Gross LT =4 TDR-Short(Other),Concentration-Gross LT =8 TDR-Long (Other),Concentration-Gross LT =8 TDR-Short(Other),Concentration-Net LT =4 TDR-Long (Other),Concentration-Net LT =4 TDR-Short (Other),Concentration-Net LT =8 TDR-Long (Other),Concentration-Net LT =8 TDR-Short (Other)
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-06,73207,5013,29846,4481,56383,13201,65878,47529,7329,25678,...,70.1,32.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-01-13,74420,4363,31707,4201,59137,11983,67702,47891,6718,26528,...,73.7,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-01-20,60716,9759,17418,5271,36129,20309,51158,42997,9558,17719,...,59.4,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-01-27,55460,10064,14914,5509,29890,19838,45464,40262,9996,15198,...,53.9,35.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-02-03,51182,8606,11388,6326,25635,18308,40567,36022,10615,15161,...,49.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-21,64756,5075,24193,1574,47905,19190,54554,44958,10202,19798,...,61.4,36.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-28,66676,4313,24576,2316,48197,21525,54826,48417,11850,18259,...,59.6,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-12-05,67817,4234,22302,4284,46494,22596,55012,49181,12805,18635,...,55.7,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
USDCHF = get_rates_w('USDCHF.a', mt5.TIMEFRAME_D1, 468)
USDCHF['close_lag1'] = USDCHF['close'].shift(1)
USDCHF

,open,high,low,close,close_lag1
time,,,,,
2022-03-01,0.92547,0.93068,0.92486,0.92930,NaN
2022-03-08,0.92800,0.94318,0.92503,0.94120,0.92930
2022-03-15,0.94039,0.94603,0.92941,0.93279,0.94120
2022-03-22,0.93288,0.93815,0.92603,0.93088,0.93279
2022-03-29,0.93086,0.93135,0.91952,0.92939,0.93088
...,...,...,...,...,...
2023-11-21,0.88352,0.88723,0.87612,0.87801,0.88360
2023-11-28,0.87771,0.87841,0.86666,0.87491,0.87801
2023-12-05,0.87465,0.88206,0.87224,0.87525,0.87491


In [77]:
cot_df = df_dict['SWISS FRANC - CHICAGO MERCANTILE EXCHANGE']
correlations = {}

# Ensure both DataFrames have the same index
cot_df = cot_df.reindex(USDCHF.index)

for col in cot_df.columns:
    # Calculate the correlation and store it in the dictionary
    correlations[col] = USDCHF['close_lag1'].corr(cot_df[col])

In [78]:
for col, val in correlations.items():
    if abs(val) > 0.5:
        print(col, val)

Commercial Positions-Short (All) -0.6833495937555332
Nonreportable Positions-Long (All) -0.7527834855293024
Nonreportable Positions-Short (All) 0.5418293429362433
Commercial Positions-Short (Old) -0.6833495937555332
Nonreportable Positions-Long (Old) -0.7527834855293024
Nonreportable Positions-Short (Old) 0.5418293429362433
% of OI-Commercial-Short (All) -0.6761023616808457
% of OI-Total Reportable-Long (All) 0.523811117486284
% of OI-Total Reportable-Short (All) -0.7560211219592428
% of OI-Nonreportable-Long (All) -0.523811117486284
% of OI-Nonreportable-Short (All) 0.7557960796285703
% of OI-Commercial-Short (Old) -0.6761023616808457
% of OI-Total Reportable-Long (Old) 0.523811117486284
% of OI-Total Reportable-Short (Old) -0.7560211219592428
% of OI-Nonreportable-Long (Old) -0.523811117486284
% of OI-Nonreportable-Short (Old) 0.7557960796285703
Concentration-Gross LT = 4 TDR-Long (All) 0.5842762245104237
Concentration-Gross LT =4 TDR-Short (All) -0.7236722035964034
Concentration-Gro